<a href="https://colab.research.google.com/github/Aravinda89/multi_label_classification/blob/main/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastai==2.0.19 fastcore==1.3.2

     |████████████████████████████████| 188 kB 8.0 MB/s 
     |████████████████████████████████| 46 kB 5.3 MB/s 
     |████████████████████████████████| 5.9 MB 64.7 MB/s 
     |████████████████████████████████| 748.8 MB 19 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:


In [ ]:
import fastai
print(fastai.__version__)

from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

2.0.19
Mounted at /content/gdrive


In [ ]:
from fastai.vision.all import *

In [ ]:
path = untar_data(URLs.PASCAL_2007)

KeyboardInterrupt: ignored

In [ ]:
path.ls()